<a href="https://colab.research.google.com/github/sridhartroy/AIML/blob/main/MapReduce_FriendRecommender_FinalS_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.5.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895798 sha256=4a66dea02174c053701859abd8aadb59c3e0d10eb1738e87c9309a0d21492ebb
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [ ]:
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder \
    .appName("MapReduce-FriendRecommender") \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
!wget https://an-ml.s3.us-west-1.amazonaws.com/soc-LiveJournal1Adj.txt

--2025-10-26 03:33:00--  https://an-ml.s3.us-west-1.amazonaws.com/soc-LiveJournal1Adj.txt
Resolving an-ml.s3.us-west-1.amazonaws.com (an-ml.s3.us-west-1.amazonaws.com)... 52.219.220.50, 52.219.117.9, 52.219.221.10, ...
Connecting to an-ml.s3.us-west-1.amazonaws.com (an-ml.s3.us-west-1.amazonaws.com)|52.219.220.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156181 (4.0M) [text/plain]
Saving to: ‘soc-LiveJournal1Adj.txt’

soc-LiveJournal1Adj 100%[===================>]   3.96M  14.4MB/s    in 0.3s    

2025-10-26 03:33:00 (14.4 MB/s) - ‘soc-LiveJournal1Adj.txt’ saved [4156181/4156181]



In [ ]:
friends = sc.textFile("soc-LiveJournal1Adj.txt")

In [ ]:
# Tiny example

lines = sc.parallelize([
    "1\t2,3,4,5,6",
    "2\t1,3,5,6",
    "3\t1,2,4,6",
    "4\t1,3,5",
    "5\t,1,2,4,6",
    "6\t1,2,3,5"
  ])


In [ ]:
lines.take(3)

['1\t2,3,4,5,6', '2\t1,3,5,6', '3\t1,2,4,6']

In [ ]:
# parse the file

def parse_line(line):
    parts = line.split("\t")
    #print(len(parts))
    user = parts[0]
    friends = parts[1].split(",") if len(parts) > 1 and parts[1] else []
    friends = [f for f in friends if f and f != user]  # drop empties/self
    return (user, friends)

In [ ]:
print("STEP 0 - Original List:")
print(friends.take(4))

adj = friends.map(parse_line).cache()
print("STEP 1 — parsed adjacency: WHO ARE MY FRIENDS?")
for x in adj.take(4): print(x)

STEP 0 - Original List:
['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94', '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592', '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878', '3\t0,12,41,55,1532,12636,13185,27552,38737']
STEP 1 — parsed adjacency: WHO ARE MY FRIENDS?
('0', ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '2

In [ ]:
inv = adj.flatMap(lambda uf: [(f, uf[0]) for f in uf[1]])
inv_grouped = inv.groupByKey().mapValues(lambda vals: sorted(set(vals)))
#print(inv.take(30))
#print(inv.groupByKey().take(30))
#print(inv_grouped.take(30))

#print(inv_grouped.sortByKey(ascending=True).take(20))

inv_grouped = inv_grouped.sortByKey(ascending=True).cache()

#print(inv_grouped.take(30))

print("\nSTEP 2 — inverted index: WHO SHARES THIS FRIEND?")
for f, users in inv_grouped.collect(): print(f, ":", users)


Streaming output truncated to the last 5000 lines.
5456 : ['13228', '18118', '21077', '21078', '2191', '25514', '27711', '2876', '29886', '31978', '31982', '31989', '3947', '42703', '42709', '42720', '42727', '42730', '42738', '42761', '42769', '45297', '48026', '7254', '9386']
5457 : ['2191', '5453', '5454', '5467']
5458 : ['14900', '15032', '19664', '21852', '2191', '25938', '25949', '26122', '26330', '27888', '2826', '29095', '29909', '37829', '41874', '4264', '4283', '43774', '43810', '45594', '5463', '7242', '8123', '8186']
5459 : ['2191', '45489', '5453', '5455', '5467', '8062']
546 : ['11207', '14351', '14928', '15309', '17356', '17441', '20567', '21164', '21206', '21273', '21869', '21935', '22532', '22549', '22551', '23234', '25913', '27334', '30896', '351', '3541', '3547', '35533', '366', '38719', '39597', '41707', '42240', '4233', '42958', '4717', '47610', '47809', '4840', '522', '560', '5766', '7611', '9284', '9453', '9892', '9928', '9951']
5460 : ['10783', '12196', '12250',

In [ ]:
from itertools import combinations

In [ ]:
# Emit all unordered user pairs per friend (+1 each)

pairs_by_dim = inv_grouped.flatMap(
    lambda fv: [((a, b), 1) for a, b in combinations(fv[1], 2)]
).cache()

print("\nSTEP 3 — pair contributions (+1 per shared friend):")
for p in pairs_by_dim.collect(): print(p)

Streaming output truncated to the last 5000 lines.
(('10043', '10085'), 1)
(('10043', '10086'), 1)
(('10043', '10087'), 1)
(('10043', '10088'), 1)
(('10043', '10089'), 1)
(('10043', '10090'), 1)
(('10043', '10091'), 1)
(('10043', '10092'), 1)
(('10043', '10093'), 1)
(('10043', '6688'), 1)
(('10043', '87'), 1)
(('10043', '9996'), 1)
(('10043', '9997'), 1)
(('10043', '9998'), 1)
(('10043', '9999'), 1)
(('10044', '10045'), 1)
(('10044', '10046'), 1)
(('10044', '10047'), 1)
(('10044', '10048'), 1)
(('10044', '10049'), 1)
(('10044', '10050'), 1)
(('10044', '10051'), 1)
(('10044', '10052'), 1)
(('10044', '10053'), 1)
(('10044', '10054'), 1)
(('10044', '10055'), 1)
(('10044', '10056'), 1)
(('10044', '10057'), 1)
(('10044', '10058'), 1)
(('10044', '10059'), 1)
(('10044', '10060'), 1)
(('10044', '10061'), 1)
(('10044', '10062'), 1)
(('10044', '10063'), 1)
(('10044', '10064'), 1)
(('10044', '10065'), 1)
(('10044', '10066'), 1)
(('10044', '10067'), 1)
(('10044', '10068'), 1)
(('10044', '10069'), 

In [ ]:
# 4) Sum contributions (this is the sparse dot-product)

mutual_counts = pairs_by_dim.reduceByKey(lambda a, b: a + b).sortByKey(ascending=True).cache()
print(mutual_counts.take(20))

print("\nSTEP 4 — mutual friend counts:")
for pair, cnt in mutual_counts.collect(): print(pair, ":", cnt)

Streaming output truncated to the last 5000 lines.
('9819', '9848') : 1
('9819', '9849') : 1
('9819', '9850') : 1
('9819', '9851') : 3
('9819', '9852') : 1
('9819', '9853') : 1
('9819', '9854') : 1
('9819', '9855') : 1
('9819', '9856') : 1
('9819', '9857') : 1
('9819', '9858') : 1
('9819', '9859') : 1
('9819', '9860') : 1
('9819', '9861') : 1
('9819', '9862') : 1
('9819', '9863') : 1
('9819', '9864') : 1
('9819', '9865') : 1
('9819', '9866') : 3
('9819', '9867') : 1
('9819', '9868') : 1
('982', '983') : 1
('982', '984') : 1
('982', '985') : 1
('982', '986') : 1
('982', '987') : 2
('982', '988') : 1
('982', '989') : 1
('982', '990') : 1
('982', '991') : 1
('982', '9912') : 1
('982', '992') : 1
('982', '993') : 1
('982', '994') : 1
('982', '995') : 1
('982', '996') : 1
('982', '997') : 1
('982', '998') : 1
('982', '999') : 1
('9820', '9821') : 2
('9820', '9822') : 3
('9820', '9823') : 1
('9820', '9824') : 2
('9820', '9825') : 3
('9820', '9826') : 1
('9820', '9827') : 1
('9820', '9828') :

In [ ]:
# Existing friends and remove

#print first the adjacency list
print("Original Adjacency : ")
print(adj.take(20))

edges_unordered = (adj
    .flatMap(lambda uf: [tuple(sorted((uf[0], f))) for f in uf[1]])
    .distinct()
    .collect()
)

edges_unordered.sort()

#print("\nExisting Friends :")
#print(edges_unordered)

b_edges = sc.broadcast(set(edges_unordered))

# print(b_edges.value)

print("\nMutual Counts :")
print(mutual_counts.take(20))

nonfriends = mutual_counts.filter(lambda kv: kv[0] not in b_edges.value).cache()

#print("\nSTEP 5 — non-friend candidate pairs (connect_score = mutuals):")
#for p in nonfriends.collect(): print(p)

Original Adjacency : 
[('0', ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94']), ('1', ['0', '5', '20', '135', '2409', '8715', '8932', '10623', '12347', '12846', '13840', '13845', '14005', '20075', '21556', '22939', '23520', '28193', '29724', '29791', '29826', '30691', '31232', '31435', '32317', '32489', '34394', '35589', '35605', '35606', '35613', '35633', '35648', '35678', '38737', '43447', '44846', '44887', '49226', '49985', '623', '629', '4999', '6156', '13912', '14248', '15190', '1763

In [ ]:
# 6) Fan-out to directed candidates

directed = nonfriends.flatMap(
    lambda kv: [
        (kv[0][0], (kv[0][1], kv[1])),  # A -> (B, score)
        (kv[0][1], (kv[0][0], kv[1]))   # B -> (A, score)
    ]
).cache()

#print("\nSTEP 6 — directed candidates (user -> (friend_to_be_recommended, connect_score)):")
#for d in directed.collect(): print(d)


In [ ]:
# 7) Group per user and take Top-10 (score desc, tie-break by ID asc)

def topk(recs, k=1):
    # recs is iterable of (candidate, score); sort by (-score, candidate)
    return [v for v, c in sorted(recs, key=lambda x: (-x[1], x[0]))[:k]]

top10 = directed.groupByKey().mapValues(lambda recs: topk(recs, 10)).cache()

print("\nSTEP 7 — top recommendations per user:")
for u, lst in top10.take(4): print(u, ":", lst)


#print(directed.groupByKey().take(20))
print(type(top10))


STEP 7 — top recommendations per user:
150 : ['127', '102', '104', '109', '110', '118', '11873', '11876', '129', '130']
19430 : ['34176', '11395', '22756', '11387', '34159', '3575', '11399', '9289', '19431', '34220']
22357 : ['19427', '19431', '22756', '11387', '13182', '22355', '16027', '23616', '34159', '11399']
29707 : ['29693', '29700', '29704', '48477', '18509', '22639', '23866', '24249', '28704', '28736']
<class 'pyspark.rdd.PipelinedRDD'>


In [ ]:
# Print All

all_users = adj.map(lambda uf: uf[0]).distinct()

#print("\nSample from all_users:")
#print(all_users.sortBy(lambda x:x).take(10)) # Print first 10 elements


#print("\nSample from top10:")
#print(top10.sortByKey(ascending=True).take(10)) # Print first 10 elements

# Collect top10 into a dictionary and broadcast it
top10_dict = dict(top10.collect())
broadcast_top10 = sc.broadcast(top10_dict)

# Map over all_users to get recommendations using the broadcasted dictionary
final = (all_users
         .map(lambda user: (user, broadcast_top10.value.get(user, [])))
         .map(lambda uv: f"{uv[0]}\t{','.join(uv[1])}").sortBy(lambda x:x))

#print("\nSTEP 8 — final lines (User\\tFriend_Rec1,Friend_Rec2,...):")
#for line in final.collect(): print(line)

#print(type(final))

In [ ]:
for k in range(1,11):
  print("Sample ", k, ": TOP 10 Recommendations for user selected based on count of mutual friends : ")
  for line in final.takeSample(False,10, seed=k): print(line)
  print("\n")

Sample  1 : TOP 10 Recommendations for user selected based on count of mutual friends : 
645	648,650,654,13168,40053,40710,41753,42039,43035,46459
28739	45545,17649,18365,22633,23958,27736,28720,45411,48281,10251
40538	40531,40546,40559,39480,40418,40548,40554,40556,40661,6655
30107	30090,30092,30094,30108,30098,30099,39193,30087,30091,30093
49518	44377,24517,23489,35408,43052,1136,16166,1660,28747,29578
9285	9257,9272,9260,44623,9246,9248,9259,9287,9293,9243
33716	5382,33717,33299,39527,7051,10792,10797,11142,13110,13412
37183	37597,44057,44066,36688,36909,37675,37701,39240,40748,44050
12639	20199,7854,1000,1014,10681,1083,1087,11095,12619,12620
2915	6978,12647,18573,12633,1797,23718,32231,8859,12650,1342


Sample  2 : TOP 10 Recommendations for user selected based on count of mutual friends : 
13382	13152,13334,13335,13336,13337,13338,13339,13340,13341,13342
39759	15353,15373,15391,18968,39763,39826,43590,8270,10620,11595
33642	14689,20273,33800,3685,39027,39045,39048
32990	23702,237

In [ ]:
for k in range(1,10):
  sample_users = final.takeSample(False, 5, seed=k)
  print(sample_users)

['9285\t9257,9272,9260,44623,9246,9248,9259,9287,9293,9243', '37604\t36994,10071,10102,18918,23273,24338,36824,36984,37091,37125', '17017\t16969,16970,239,1433,14892,16883,16902,16911,16932,16938', '7140\t7127,7136,7138,7148,7149,7150,7165,10287,14495,14497', '30107\t30090,30092,30094,30108,30098,30099,39193,30087,30091,30093']
['3439\t3434,25319,3391,3392,3393,3394,3395,3396,3397,3398', '25538\t25564,15072,15076,25541,25556,25557,25589,25602,25611,10413', '11390\t11383,11385,11387,11391,11392,11404,11393,11394,11395,11396', '32911\t32942,32912,32913,32900,32904,32872,32922,32920,32861,32876', '33642\t14689,20273,33800,3685,39027,39045,39048']
['18793\t18798,18765,18764,18796,18799,18760,18761,18763,18776,18792', '35989\t35999,36039,36009,35992,36013,17748,36017,36025,35985,36002', '7571\t7583,5653,7590,7619,2786,7517,7530,7545,7589,1266', '34758\t14242,19427,19431,8677,9289,11387,11410,14047,14066,16030', '11607\t11604,11615,11616,11618,11622,11623,11627,11639,11602,11624']
['39912\t1